In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
import collections
import csv
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from string import punctuation
from nltk.stem import WordNetLemmatizer
from autocorrect import Speller
wordnet_lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
spell = Speller(lang='en')
ps = PorterStemmer()
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
import matplotlib.image as mpimg
from scipy import misc
from nltk.text import TextCollection

In [2]:
cd '/Users/chenjiawen'

/Users/chenjiawen


In [44]:
def list_dictionary(text):
    list0=[]
    if text != '':
        for key,value in eval(text).items():
            key = key.lower()
            if(isinstance(eval(value),dict)):
                for key1,value1 in eval(value).items():
                    value1 = str(value1).lower()
                    value1 = value1.replace('u\'',"")
                    value1 = value1.replace('\'',"") 
                    if value1 != "false" and value1 != "none" and value1 != "no":
                        if  value1 == "true" or value1 == "yes":
                            key1=key1.lower()
                            list0.append(key+"_"+key1+"+yes")
                        else:                      
                            list0.append(key+"_"+key1+"+"+value1)
                    else:
                        key = key.lower()
                        key1=key1.lower()
                        list0.append(key+"_"+key1+"+"+"none")
                continue
            value = str(value).lower()
            value = value.replace('u\'',"")
            value = value.replace('\'',"")
            if value != "false" and value != "none" and value != "no": 
                if value == "true" or value == "yes":
                    list0.append(key+"+yes")
                else:                  
                    list0.append(key+"+"+value)
            else:
                list0.append(key+"+none")
        return(list0)
    else:    
        return(list0)

In [45]:
def Convert(lst): 
    res_dct = {lst[i]: lst[i + 1] for i in range(0, len(lst), 2)} 
    return res_dct

In [46]:
business=[]
with open('business_WI.csv', newline='') as f:
    temp = csv.reader(f)
    for row in temp:
        business.append(row)
name=['column','business_id','name','address','city','state','postal_code','latitude','longitude','stars','review_count','is_open','attributes','categories','hours']
business=pd.DataFrame(columns=name,data=business)
business = business[1:]

In [47]:
list_dictionary(business['attributes'][1])

['businessparking_garage+none',
 'businessparking_street+none',
 'businessparking_validated+none',
 'businessparking_lot+none',
 'businessparking_valet+none',
 'restaurantsgoodforgroups+none',
 'wifi+free',
 'restaurantstakeout+yes',
 'restaurantsattire+casual',
 'businessacceptscreditcards+yes',
 'restaurantsdelivery+none',
 'goodforkids+none',
 'restaurantsreservations+none',
 'restaurantspricerange2+1',
 'goodformeal_dessert+none',
 'goodformeal_latenight+none',
 'goodformeal_lunch+none',
 'goodformeal_dinner+none',
 'goodformeal_brunch+none',
 'goodformeal_breakfast+none']

In [48]:
for index,bus in business.iterrows():
    bus['attributes'] = list_dictionary(bus['attributes'])
    #rev = stopwords_stemming(replace_to(rev))

In [49]:
business.to_json('business_final.json',orient='records')

In [50]:
list_bus = []
    
with open('business_final.json','r') as load_f:
    list_bus = json.load(load_f)

In [51]:
attributes = []
for items in business["attributes"]:
    attributes.append(items)

In [52]:
def expand_list(nested_list):
    for item in nested_list:
        if isinstance(item, (list, tuple)):
            for sub_item in expand_list(item):
                yield sub_item
        else:
            yield item


In [53]:
attributes = list(expand_list(attributes))

In [72]:
att_uni = np.unique(attributes)

In [73]:
att_uni

array(['alcohol+beer_and_wine', 'alcohol+full_bar', 'alcohol+none',
       'ambience+none', 'ambience_casual+none', 'ambience_casual+yes',
       'ambience_classy+none', 'ambience_classy+yes',
       'ambience_divey+none', 'ambience_divey+yes',
       'ambience_hipster+none', 'ambience_hipster+yes',
       'ambience_intimate+none', 'ambience_intimate+yes',
       'ambience_romantic+none', 'ambience_romantic+yes',
       'ambience_touristy+none', 'ambience_touristy+yes',
       'ambience_trendy+none', 'ambience_trendy+yes',
       'ambience_upscale+none', 'ambience_upscale+yes', 'bestnights+none',
       'bestnights_friday+none', 'bestnights_friday+yes',
       'bestnights_monday+none', 'bestnights_monday+yes',
       'bestnights_saturday+none', 'bestnights_saturday+yes',
       'bestnights_sunday+none', 'bestnights_sunday+yes',
       'bestnights_thursday+none', 'bestnights_thursday+yes',
       'bestnights_tuesday+none', 'bestnights_tuesday+yes',
       'bestnights_wednesday+none', 'b

In [74]:
for i in range(len(list_bus)):
    record=list_bus[i]
    for word in att_uni:
        if word not in record['attributes']:
            record['attributes'].append(word+"+NA")

In [57]:
restaurantsattire=['restaurantsattire_casual',
       'restaurantsattire_dressy', 'restaurantsattire_formal']

In [58]:
for i in range(len(list_bus)):
    record=list_bus[i]
    count =0
    for word in restaurantsattire:
        if word in record['attributes']:
            count = count +1
    if count ==0:
        record['attributes'].append("restaurantsattire+NA")

In [59]:
goodformeal=['goodformeal_breakfast',
       'goodformeal_brunch', 'goodformeal_dessert', 'goodformeal_dinner',
       'goodformeal_latenight', 'goodformeal_lunch']

In [60]:
for i in range(len(list_bus)):
    record=list_bus[i]
    count =0
    for word in goodformeal:
        if word in record['attributes']:
            count = count +1
    if count ==0:
        record['attributes'].append("goodformeal+NA")

In [61]:
best_night = ['bestnights_friday',
       'bestnights_monday', 'bestnights_saturday', 'bestnights_sunday',
       'bestnights_thursday', 'bestnights_tuesday',
       'bestnights_wednesday']

In [62]:
for i in range(len(list_bus)):
    record=list_bus[i]
    count =0
    for word in best_night:
        if word in record['attributes']:
            count = count +1
    if count ==0:
        record['attributes'].append("bestnights+NA")

In [63]:
wifi =['wifi+free', 'wifi+paid']

In [30]:
for i in range(len(list_bus)):
    record=list_bus[i]
    count =0
    for word in wifi:
        if word in record['attributes']:
            count = count +1
    if count ==0:
        record['attributes'].append("wifi+NA")

In [31]:
music=['music_background_music', 'music_dj', 'music_jukebox',
       'music_karaoke', 'music_live', 'music_video']

In [32]:
for i in range(len(list_bus)):
    record=list_bus[i]
    count =0
    for word in music:
        if word in record['attributes']:
            count = count +1
    if count ==0:
        record['attributes'].append("music_NA")

In [33]:
parking  = ['bikeparking', 'businessparking_garage',
       'businessparking_lot', 'businessparking_street',
       'businessparking_valet', 'businessparking_validated']

In [34]:
for i in range(len(list_bus)):
    record=list_bus[i]
    count =0
    for word in parking:
        if word in record['attributes']:
            count = count +1
    if count ==0:
        record['attributes'].append("parking_none")

In [75]:
my_df = pd.DataFrame(list_bus)
my_df.to_csv('bussiness.csv', index=False, header=False)

In [29]:
list_bus[0]["attributes"]

['wifi_free',
 'restaurantstakeout',
 'restaurantsattire_casual',
 'businessacceptscreditcards',
 'restaurantspricerange2_1',
 'alcohol_beer_and_wine_none',
 'alcohol_full_bar_none',
 'caters_none',
 'coatcheck_none',
 'dietaryrestrictions_vegan_none',
 'dietaryrestrictions_vegetarian_none',
 'dogsallowed_none',
 'drivethru_none',
 'goodfordancing_none',
 'goodforkids_none',
 'happyhour_none',
 'hastv_none',
 'outdoorseating_none',
 'restaurantsdelivery_none',
 'restaurantsgoodforgroups_none',
 'restaurantsreservations_none',
 'restaurantstableservice_none',
 'smoking_outdoor_none',
 'smoking_none',
 'wheelchairaccessible_none',
 'businessacceptsbitcoin_none',
 'goodformeal_none',
 'bestnights_none',
 'music_none',
 'parking_none']

In [37]:
attributes=[]
for i in range(len(list_bus)):
    attributes.append(list_bus[i]["attributes"])
attributes = list(expand_list(attributes))

In [38]:
len(business)

1703

In [39]:
import re
from collections import Counter

In [40]:
Counter(business['stars'])

Counter({'2.5': 177,
         '4.0': 464,
         '3.5': 428,
         '3.0': 281,
         '4.5': 189,
         '1.0': 15,
         '2.0': 82,
         '1.5': 33,
         '5.0': 34})

In [41]:
Counter(attributes).most_common(100)

[('caters_NA', 1703),
 ('coatcheck_NA', 1703),
 ('dietaryrestrictions_vegan_NA', 1703),
 ('dietaryrestrictions_vegetarian_NA', 1703),
 ('dogsallowed_NA', 1703),
 ('drivethru_NA', 1703),
 ('goodfordancing_NA', 1703),
 ('goodforkids_NA', 1703),
 ('happyhour_NA', 1703),
 ('hastv_NA', 1703),
 ('outdoorseating_NA', 1703),
 ('restaurantsdelivery_NA', 1703),
 ('restaurantsgoodforgroups_NA', 1703),
 ('restaurantsreservations_NA', 1703),
 ('restaurantstableservice_NA', 1703),
 ('restaurantstakeout_NA', 1703),
 ('smoking_NA', 1703),
 ('wheelchairaccessible_NA', 1703),
 ('businessacceptsbitcoin_NA', 1703),
 ('businessacceptscreditcards_NA', 1703),
 ('goodformeal_NA', 1703),
 ('bestnights_NA', 1703),
 ('music_NA', 1703),
 ('parking_none', 1703),
 ('smoking_outdoor_NA', 1668),
 ('businessacceptscreditcards_yes', 1514),
 ('alcohol_beer_and_wine_NA', 1466),
 ('restaurantstakeout_yes', 1421),
 ('restaurantsattire_casual', 1418),
 ('businessparking_validated_none', 1414),
 ('businessparking_valet_none'

In [400]:
def countnum(word):
    count_num=[0,0,0,0,0]
    for i in range(len(list_bus)):
        review=list_bus[i]
        if word in review['attributes']:
            point=float(review['stars'])
            if point <= 1:
                count_num[0] += 1
            elif point <= 2:
                count_num[1] += 1
            elif point <= 3:
                count_num[2] += 1
            elif point <= 4:
                count_num[3] += 1
            elif point <= 5:
                count_num[4] += 1
    return(count_num)

In [401]:
def count_res(list0):
    dinner_star = [0,0,0,0,0]
    for i in range(len(list0)):
            review=list0[i]
            point=float(review['stars'])
            if point <= 1:
                dinner_star[0] += 1
            elif point <= 2:
                dinner_star[1] += 1
            elif point <= 3:
                dinner_star[2] += 1
            elif point <= 4:
                dinner_star[3] += 1
            elif point <= 5:
                dinner_star[4] += 1
    return(dinner_star)

In [1]:
dinner_star

NameError: name 'dinner_star' is not defined

In [402]:
dinner_star = count_res(list_bus)

In [405]:
countnum("businessacceptscreditcards_none")

[4, 15, 41, 81, 48]

In [407]:
word_list=['noiselevel_average',
           'noiselevel_loud', 'noiselevel_quiet', 'noiselevel_very_loud','bestnights_friday',
           'bestnights_monday', 'bestnights_saturday', 'bestnights_sunday',
           'bestnights_thursday', 'bestnights_tuesday',
           'bestnights_wednesday','byobcorkage_yes_corkage', 'byobcorkage_yes_free','coatcheck',
          'wifi_none','wifi_free','music_none','bestnights_none','outdoorseating_none','outdoorseating',
          'restaurantsdelivery_none','restaurantsdelivery','businessacceptscreditcards_none','businessacceptscreditcards',
          'parking_none','businessparking_street']
def savefigure(wordlist):
    for i in range(len(wordlist)):
        word=wordlist[i]
        word_count=countnum(word)
        fig, ax = plt.subplots()
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        percentage = [0,0,0,0,0]
        for h in range(5):
            percentage[h] =  word_count[h]/dinner_star[h]
        plt.bar(range(1,6), percentage, facecolor = '#9ca8b8',edgecolor = 'white')
        plt.xlabel("Stars")
        plt.ylabel("Percentage")
        plt.title("The Number of restaurant with "+'attribute: "'+word+'"')
        plt.xticks( range(1,6), ('0~1', '1~2', '2~3', '3~4', '4~5') )
        j=0
        for x,y in zip(range(1,6), percentage):
    #ha : horizontal alignment
    #va : vertical alignment
            plt.text(x + 0.01,y+0.00005,'%d'%word_count[j],ha = 'left',va='bottom',color = "r")
            plt.text(x + 0.01,y+0.00005,'%d%%'%(y*100),ha = 'right',va='bottom', color = "green")
            j=j+1
        plt.savefig(word+'.png')
        plt.close()
savefigure(word_list)
